In [45]:
import torch
import tqdm

## Compare

In [52]:
model = 'activations_pythia_410m/nat_lang/model-activations/EleutherAI_pythia-410m_15L_68.pt'
sae = 'activations_pythia_410m/nat_lang/sae-activations/EleutherAI_sae-pythia-410m-65k_15L_68.pt'
#model = 'activations_pythia_160m/nat_lang/model-activations/EleutherAI_pythia-160m_11L_19.pt'
#sae = 'activations_pythia_160m/nat_lang/sae-activations/EleutherAI_sae-pythia-160m-32k_11L_19.pt'


checkpoint_model = torch.load(model, map_location=torch.device('cuda:0'), weights_only=False)
print(len(checkpoint_model))
print(type(checkpoint_model))
print(checkpoint_model[3])
print(checkpoint_model[4])
print(checkpoint_model[5])

print('_'*25)
checkpoint_sae = torch.load(sae, map_location=torch.device('cuda:0'), weights_only=False)
checkpoint_sae = [c.to_dense() if c is not None else None for c in checkpoint_sae]
print(len(checkpoint_sae))
print(type(checkpoint_sae))
print(checkpoint_sae[3])
print(checkpoint_sae[4])
print(checkpoint_sae[5])

print('_'*25)
# Tallies?
assert len(checkpoint_sae) == len(checkpoint_model), "Lengths dont match"
for i in tqdm.tqdm(range(len(checkpoint_sae))):
    if checkpoint_sae[i] is not None:
        assert checkpoint_sae[i].shape[0] == checkpoint_sae[i].shape[0], f"Positions dont match for index {i}"
        non_zero = (checkpoint_sae[i] != 0).sum(dim = -1)
        assert non_zero.max() == 32, "More than 32 non zeros"
        assert non_zero.min() == 32, "Less than 32 non zeros"

print('_'*25)
assert sum([1 if c is None else 0 for c in checkpoint_sae]) == sum([1 if c is None else 0 for c in checkpoint_model]), "Unequal Number of Nones"

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory